In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import time 
import re

In [3]:
#path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
#sets the pathway for chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome',**executable_path)

# Visit the NASA Mars News Site 

In [ ]:
#visit mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

#to make a delay when loading the page.
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

In [ ]:
#Convert browser html to a soup object and then quit the browser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [ ]:
slide_elem.find("div", class_='content_title')

In [ ]:
#Use parent element to find firest tag and save it as the title.
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

In [ ]:
#Use parent element to find the paragrapht text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

# JPL Space Images Featured Image

In [ ]:
#Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
#drill down, find and click the full. image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()


In [ ]:
#Find the more info button and click - note that there is white space **
browser.is_element_present_by_text('more info    ',wait_time=1)

In [ ]:
#another way to perform this is finding partial text
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [ ]:
#parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

In [ ]:
#use the base url to creat an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

# Mars Weather

In [ ]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

time.sleep(5)

html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#Find a tweet with the data-name "Mars Weather"
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name":"Mars Weather"})


In [ ]:
mars_weather_tweet.find("p", "tweet-text").get_text()

In [ ]:
#Due to twitter changes try searching within the tweet for the ptag or span tag
#containgn the tweet text.f
#text using a regular exptression pattern that includes the string 'sol'
#if there is no p tag with a class of 'tweet-text'
try:
    mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
    mars_weather
except AttributeError:
    
    pattern = re.compile(r'InSight sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
    
mars_weather

# Mars Hemispheres

In [5]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)



In [6]:

#html = browser.html
#weather_soup = BeautifulSoup(html, 'html.parser')    

In [9]:
hemisphere_image_urls = []

# get a list of all the hemispheres
links = browser.find_by_css("a.product-item h3")
print(links)
print(len(links))

#Loop through links, click links, find the sample anchor return the href
for i in range(len(links)):
    hemisphere = {}
    
    #find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    #find the sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    print(sample_elem)
    hemisphere['img_url'] = sample_elem['href']
    
    #Get hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    #Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    #navigate back
    browser.back()

4


/Users/eli/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [10]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

# Mars Facts

In [11]:
import pandas as pd
df = pd.read_html("https://space-facts.com/mars/", index_col=None)[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [12]:
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [13]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [14]:
browser.quit()

In [ ]:
def scrape_all():
    browser = Browser("chrome", executable_path="chromedriver", headless=False)
    news_title, news_paragraph = mars_news(browser)

In [ ]:
data = {
    "news_title": news_title,
    "news_paragraph": news_paragraph,
    "featured_image": featured_image(browser),
    "hemispheres": hemispheres(browser),
    "facts": mars_facts(),
    "last_modified": dt.datetime.now()     
}

In [ ]:
browser.quit()
return data

In [ ]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
def scrape():
    browser = init_browser()
    listings = {}

    url = "https://raleigh.craigslist.org/search/hhh?max_price=1500&availabilityMode=0"
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    listings["headline"] = soup.find("a", class_="result-title").get_text()
    listings["price"] = soup.find("span", class_="result-price").get_text()
    listings["hood"] = soup.find("span", class_="result-hood").get_text()

    return listings

In [ ]:
print("Hello")